# Explainability Metrics


### Algorithm Class Score 

In [26]:
from difflib import get_close_matches
import tensorflow as tf
import torch.nn as nn

def algorithm_class_score(model):
    """Returns an explainability score based on the model class. More complex models, will have a lower score.
        :param model: model to be tested
        :return: normalized score of [1, 5]
    """

    alg_score = {
    "RandomForestClassifier": 4,
    "KNeighborsClassifier": 3,
    "SVC": 2,
    "GaussianProcessClassifier": 3,
    "DecisionTreeClassifier": 5,
    "MLPClassifier": 1,
    "AdaBoostClassifier": 3,
    "GaussianNB": 3.5,
    "QuadraticDiscriminantAnalysis": 3,
    "LogisticRegression": 4,
    "LinearRegression": 3.5,
    }

    model_name = type(model).__name__

    # Check if the model_name is in the dictionary
    if model_name in alg_score:
        exp_score = alg_score[model_name]
        return exp_score 

    # Check if the model is a Neural Network
    if isinstance(model, tf.keras.Model) or isinstance(model, tf.Module) or isinstance(model, nn.Module):
        return 1
    
    # If not, try to find a close match
    close_matches = get_close_matches(model_name, alg_score.keys(), n=1, cutoff=0.6)
    if close_matches:
        exp_score = alg_score[close_matches[0]]
        return exp_score
    
    # If no close match found 
    print(f"No matching score found for '{model_name}'")
    return None

In [2]:
# Example Decision Tree Classifer and Regressor
from sklearn import tree

Classifier = tree.DecisionTreeClassifier()
Regressor = tree.DecisionTreeRegressor()

print(type(Classifier).__name__)
print(algorithm_class_score(Classifier))

print(type(Regressor).__name__)
print(algorithm_class_score(Regressor))

DecisionTreeClassifier
5
DecisionTreeRegressor
5


In [3]:
# Example Neural Network Tensorflow 
import tensorflow as tf

TFNN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_dim=128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

print(type(TFNN).__name__)
print(algorithm_class_score(TFNN))

Sequential
1


c:\Users\anton\anaconda3\envs\INV\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Custom non-sequential NN using keras
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation="relu")
        self.dense2 = tf.keras.layers.Dense(5, activation="softmax")
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dropout(x, training=training)
        return self.dense2(x)

model = MyModel()

print(type(model).__name__)
print(algorithm_class_score(model))

MyModel
1


In [4]:
# Example Neural Network Pytoch
import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(128, 64)  
        self.fc2 = nn.Linear(64, 32)  
        self.fc3 = nn.Linear(32, 1)    

    def forward(self, x):
        x = torch.relu(self.fc1(x))   
        x = torch.relu(self.fc2(x))    
        x = torch.sigmoid(self.fc3(x)) 
        return x
    
TOCHNN = NeuralNetwork()

print(type(TOCHNN).__name__)
print(algorithm_class_score(TOCHNN))

NeuralNetwork
1


### Feature Correlation Score

In [10]:
import numpy as np
import pandas as pd

# The higher the score, the smaller the percentage of features with hight coorelation in relation to the average coorelation 
def correlated_features_score(train_data, test_data, thresholds=[0.05, 0.16, 0.28, 0.4], target_column=None, verbose=False):
    print(type(test_data))
    if type(test_data) != 'pandas.core.frame.DataFrame':
        test_data = pd.DataFrame(test_data)

    test_data = test_data.copy()
    train_data = train_data.copy()
     
    if target_column:
        X_test = test_data.drop(target_column, axis=1)
        X_train = train_data.drop(target_column, axis=1)
    else:
        X_test = test_data.iloc[:,:-1]
        X_train = train_data.iloc[:,:-1]
        
    
    df_comb = pd.concat([X_test, X_train])
    df_comb = df_comb._get_numeric_data()
    corr_matrix = df_comb.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    
    # Compute average and standar deviation from upper correlation matrix 
    avg_corr = upper.values[np.triu_indices_from(upper.values,1)].mean()
    std_corr = upper.values[np.triu_indices_from(upper.values,1)].std()

    # Find features with correlation greater than avg_corr + std_corr
    to_drop = [column for column in upper.columns if any(upper[column] > (avg_corr+std_corr))]
    if verbose: print(f"Removed features: {to_drop}")
    
    pct_drop = len(to_drop)/len(df_comb.columns)
    
    bins = thresholds
    score = 5-np.digitize(pct_drop, bins, right=True) 
    
    return score

For experimental purposes there will be used the following datasets:

- [Healthcare Diabetes Dataset](https://www.kaggle.com/datasets/nanditapore/healthcare-diabetes)
- [Iris Dataset](https://www.kaggle.com/datasets/uciml/iris)

In [11]:
# Example with Healthcare Diabetes Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

health = pd.read_csv('Data/Healthcare-Diabetes.csv')

health_X = health[health.columns[1:9]]
health_y = health[health.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(health_X, health_y, test_size=0.33, random_state=42)

print(correlated_features_score(X_train, X_test, verbose=True))

<class 'pandas.core.frame.DataFrame'>
Removed features: ['Insulin', 'BMI']
2


C:\Users\anton\AppData\Local\Temp\ipykernel_2716\191569567.py:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [5]:
# Example with Iris Dataset
iris = pd.read_csv('Data/iris.csv')

iris_X = iris[iris.columns[:5]]
iris_y = iris['class']

X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.33, random_state=42)
print(correlated_features_score(X_train, X_test, verbose=True))


Removed features: ['petallength']
3


C:\Users\anton\AppData\Local\Temp\ipykernel_9748\3473009596.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


### Model Size Score

In [6]:
import numpy as np

# Returns a score based on the number of attributes(columns) in the dataset
def model_size_score(test_data, thresholds = np.array([10,30,100,500])):
    print(test_data.shape)
    dist_score = 5- np.digitize(test_data.shape[1]-1 , thresholds, right=True) # -1 for the id?
    
    return dist_score

print(model_size_score(iris_X))

(150, 5)
5


In [142]:
def model_size(model, test_dataset=None):
    """
    Calculate the size of a machine learning model based on its type.
    
    Parameters:
    model : object
        The machine learning model whose size needs to be determined.
    test_dataset : array-like, optional
        A test dataset to be used for certain types of models.

    Returns:
    int or str or None
        The size of the model, which can be the number of parameters, 
        the number of nodes, the number of support vectors, or the 
        number of features seen in fit, depending on the model type.
    """
    
    # If the model is a TensorFlow Keras Model, return the count of parameters
    if isinstance(model, tf.keras.Model):
        return model.count_params()
    
    # If the model is a TensorFlow Module or PyTorch Module, return the sum of parameters
    elif isinstance(model, tf.Module) or isinstance(model, nn.Module):
        return sum(p.numel() for p in model.parameters())
    
    # If the model has 'estimators_', typically an ensemble model like RandomForest
    elif hasattr(model, 'estimators_'):
        count = 0
        for i, est in enumerate(model.estimators_):
            # If the estimator has a tree structure, add the number of nodes
            if hasattr(est, 'tree_'):
                count += est.tree_.node_count
            # If the estimator has support vectors, add their count
            elif hasattr(est, 'n_support_'):
                count += sum(est.n_support_)
        return count
    
    # If the model is a Support Vector Classifier
    elif hasattr(model, 'SVC'):
        return sum(model.n_support_)

    # If the model has a tree structure, return the number of nodes
    elif hasattr(model, 'tree_'):
        return model.tree_.node_count
    
    # Return the number of features seen during fit if applicable
    elif hasattr(model, 'n_features_in_'):
        return 'n_features_in_'
    
    # If a test dataset is provided, return the number of features in the dataset
    elif test_dataset is not None:
        return test_dataset.shape[1]
    
    # If none of the above conditions are met, return None
    else:
        return None

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
rf = RandomForestClassifier(random_state=0)
rf.fit(X, y)

print(model_size(rf))

8976


In [44]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import load_iris

# Load a sample dataset
data = load_iris()
X, y = data.data, data.target

# Train an AdaBoost Classifier with default base estimator (decision stumps)
ada = AdaBoostClassifier()
ada.fit(X, y)

print(model_size(ada))

150


In [143]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = BaggingClassifier(estimator=SVC(),
                        n_estimators=10, random_state=0).fit(X, y)

count = 0
for i in range(len(clf.estimators_)):
    if hasattr(clf.estimators_[i], 'n_support_'):
        count += sum(clf.estimators_[i].n_support_)
print(count)

print(hasattr(clf.estimators_[1], 'n_support'))

print(clf.estimators_[1].n_support_)

print(model_size(clf))

374
False
[20 19]
a
a
a
a
a
a
a
a
a
a
374


In [128]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

# Step 1: Load and preprocess the data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize the images to [0, 1] range
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Step 2: Create a simple neural network model
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input image
    Dense(128, activation='relu'),  # First hidden layer
    Dense(10, activation='softmax') # Output layer
])

# Step 3: Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 4: Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Step 5: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


c:\Users\anton\anaconda3\envs\INV\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8635 - loss: 0.4705 - val_accuracy: 0.9523 - val_loss: 0.1640
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9598 - loss: 0.1393 - val_accuracy: 0.9672 - val_loss: 0.1122
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9734 - loss: 0.0870 - val_accuracy: 0.9720 - val_loss: 0.0974
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9825 - loss: 0.0605 - val_accuracy: 0.9712 - val_loss: 0.0985
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9867 - loss: 0.0455 - val_accuracy: 0.9748 - val_loss: 0.0878
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9733 - loss: 0.0883
Test accuracy: 0.9765999913215637


In [132]:
model_size(model, X_test)

101770

In [29]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)

dt.fit(X, y)

print(model_size(dt))


101


### Feature Relevance Score

In [11]:
from difflib import get_close_matches
import numpy as np

def feature_relevance_score(model, thresholds = [0.05, 0.1, 0.2, 0.3]):

    scale_factor = 1.5 
    distri_threshold = 0.5

    regression = ['LogisticRegression', 'LogisticRegression']
    classifier = ['RandomForestClassifier', 'DecisionTreeClassifier']

    # Feature Importance for Regressions 
    if (type(model).__name__ in regression) or (get_close_matches(type(model).__name__, regression, n=1, cutoff=0.6)): 
        importance = model.coef_.flatten()

        total = 0
        for i in range(len(importance)):
            total += abs(importance[i])

        for i in range(len(importance)):
            importance[i] = abs(importance[i]) / total

    # Feature Importance fo Random Forest, model needs to be fitted
    elif  (type(model).__name__ in classifier) or (get_close_matches(type(model).__name__, classifier, n=1, cutoff=0.6)):
        importance = model.feature_importances_
   
    else:
        return None

    # absolut values
    importance = importance
    indices = np.argsort(importance)[::-1] # indice of the biggest value in the importance list
    importance = importance[indices]
    
    # calculate quantiles for outlier detection
    q1, q3 = np.percentile(importance, [25,75])
    lower_threshold , upper_threshold = q1 - scale_factor*(q3-q1),  q3 + scale_factor*(q3-q1) 
    
    # percentage of features that concentrate distri_threshold percent of all importance
    pct_dist = sum(np.cumsum(importance) < distri_threshold) / len(importance)
    
    score = np.digitize(pct_dist, thresholds, right=False) + 1 
    return score


In [12]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

for i in range(2,11,2):
    X, y = make_classification(n_samples=1000, n_features=10, n_informative=i, n_redundant=0, n_repeated=0, n_clusters_per_class=2, n_classes=2, random_state=42)
    model = RandomForestClassifier(random_state=123)
    model.fit(X,y)

    print(feature_relevance_score(model))

1
4
4
5
5


In [107]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

for i in range(2,11,2):
    X, y = make_classification(n_samples=1000, n_features=10, n_informative=i, n_redundant=0, n_repeated=0, n_clusters_per_class=2, n_classes=2, random_state=42)
    model = LogisticRegression()
    model.fit(X,y)
    
    print(feature_relevance_score(model))

0.0
0.1
0.1
0.2
0.2


In [108]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

for i in range(2,11,2):
    X, y = make_regression(n_samples=1000, n_features=10, n_informative=i, n_targets=1, random_state=123)
    model = LinearRegression()
    model.fit(X,y)
    
    print(feature_relevance_score(model))


0.0
0.1
0.1
0.2
0.3


### Feature Importance

In [ ]:
import shap
import math
from scipy.stats import variation

def get_feature_importance_cv(test_sample, model, cfg):
    """Calculates feature importance coefficient of variation
       :param test_sample: one test sample
       :param model: the model
       :param cfg: configs
       :return: the coefficient of variation of the feature importance scores, [0, 1]
    """
    cv = 0
    batch_size = cfg['batch_size']
    device = cfg['device']
    if isinstance(model, torch.nn.Module):
        batched_data, _ = test_sample

        n = batch_size
        m = math.floor(0.8 * n)

        background = batched_data[:m].to(device)
        test_data = batched_data[m:n].to(device)

        e = shap.DeepExplainer(model, background)
        shap_values = e.shap_values(test_data)
        if shap_values is not None and len(shap_values) > 0:
            sums = np.array([shap_values[i].sum() for i in range(len(shap_values))])
            abs_sums = np.absolute(sums)
            cv = variation(abs_sums)
    return cv



# -> agnostic 

In [1]:
import shap
import numpy as np

from scipy.stats import variation
import shap
import pandas as pd
from sklearn.datasets import make_classification, make_moons
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

from sklearn.linear_model import LogisticRegression

# Assuming model is your trained model
# X_train is your training dataset
# X_explain is the dataset you want to explain

# Define the prediction function
def predict(model): 
    return model.predict_proba if hasattr(model, 'predict_proba') else model.predict


def feature_importance(model, test_data):
    # Create a background dataset (subset of your training data)
    background = shap.sample(test_data, 100)  # Using 100 samples for background

    # Initialize KernelExplainer with the prediction function and background dataset
    explainer = shap.KernelExplainer(predict(model), background)

    # Calculate SHAP values for the dataset you want to explain
    shap_values = explainer.shap_values(X.iloc[0])
  
    # calculare variance of absolute shap values 
    if shap_values is not None and len(shap_values) > 0:
        sums = np.array([abs(shap_values[i]).sum() for i in range(len(shap_values))])
        print(sums)
        cv = np.std(sums) / np.mean(sums)
        return cv


for i in range(4,11,2):
    X, y = make_regression(n_samples=1000, n_features=10, n_informative=i, n_targets=1, random_state=123)
    model = LinearRegression()

    #X, y = make_classification(n_samples=1000, n_features=10, n_informative=i,flip_y=0, n_redundant=0, n_repeated=0, n_clusters_per_class=2, n_classes=5, random_state=42)
    #model = RandomForestClassifier(random_state=123)
    
    model.fit(X,y)
    X = pd.DataFrame(X)
    print(feature_importance(model, X))

#deep modelssss and regression 

c:\Users\anton\anaconda3\envs\INV\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[   0.            0.          -26.3898406     0.            0.
    0.         -173.96069793  -36.47846048  178.36556706    0.        ]
[  0.           0.          26.3898406    0.           0.
   0.         173.96069793  36.47846048 178.36556706   0.        ]
1.6488185941212234
[   0.            7.95771464   -7.50706771    0.            0.
    0.           32.69152363    4.95209513  -20.46210449 -112.74815127]
[  0.           7.95771464   7.50706771   0.           0.
   0.          32.69152363   4.95209513  20.46210449 112.74815127]
1.7697208040239418
[ 139.29544059   15.62943679   10.79985054    0.          107.32512664
    0.          -76.94435297    4.72924418 -106.35838137  160.60253257]
[139.29544059  15.62943679  10.79985054   0.         107.32512664
   0.          76.94435297   4.72924418 106.35838137 160.60253257]
0.9608445625198315
[  41.69635121 -143.00262003   25.77577464  -30.38333323   27.94207704
   28.00904702  -15.18115511   -0.86557109   90.38688799   17.81065737]
[ 41

In [6]:
# Custom non-sequential NN using keras
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(32, activation="relu")
        self.dense2 = tf.keras.layers.Dense(5, activation="softmax")
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dropout(x, training=training)
        return self.dense2(x)

model = MyModel()

print(type(model).__name__)
print(algorithm_class_score(model))

MyModel
1


In [5]:
def model_size_score(model, dataset):
    # add comment
    if isinstance(model, tf.keras.Model):
        return (model.count_params())
    elif isinstance(model, tf.Module) or isinstance(model, nn.Module):
        return (sum(p.numel() for p in model.parameters()))
    else:
        return (dataset.shape[1]-1)

In [8]:
from sklearn2pmml.util import deep_sizeof

print(model.__sizeof__())
#print(deep_sizeof(model, with_overhead=True, verbose=True))

32


In [9]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor

X, y = make_regression(n_samples = 10000, n_features = 10)

estimator = RandomForestRegressor(n_estimators = 31, random_state = 13)
estimator.fit(X, y)

print("Initial state: {} B".format(estimator.__sizeof__()))

estimator.fit(X, y)

print("Final fitted state: {} B".format(estimator.__sizeof__()))

Initial state: 32 B
Final fitted state: 32 B


In [12]:
import numpy
import types

def is_instance_attr(obj, name):
  if not hasattr(obj, name):
    return False
  if name.startswith("__") and name.endswith("__"):
    return False
  v = getattr(obj, name)
  if isinstance(v, (types.BuiltinFunctionType, types.BuiltinMethodType, types.FunctionType, types.MethodType)):
    return False
  # See https://stackoverflow.com/a/17735709/
  attr_type = getattr(type(obj), name, None)
  if isinstance(attr_type, property):
    return False
  return True

def get_instance_attrs(obj):
  names = dir(obj)
  names = [name for name in names if is_instance_attr(obj, name)]
  return names

def deep_sklearn_sizeof(obj, verbose = True):
  # Primitive type values
  if obj is None:
    return obj.__sizeof__()
  elif isinstance(obj, (int, float, str, bool, numpy.int64, numpy.float32, numpy.float64)):
    return obj.__sizeof__()
  # Iterables
  elif isinstance(obj, list):
    sum = [].__sizeof__() # Empty list
    for v in obj:
      v_sizeof = deep_sklearn_sizeof(v, verbose = False)
      sum += v_sizeof
    return sum
  elif isinstance(obj, tuple):
    sum = ().__sizeof__() # Empty tuple
    for i, v in enumerate(obj):
      v_sizeof = deep_sklearn_sizeof(v, verbose = False)
      sum += v_sizeof
    return sum
  # Numpy ndarrays
  elif isinstance(obj, numpy.ndarray):
    sum = obj.__sizeof__() # Array header
    sum += (obj.size * obj.itemsize) # Array content
    return sum
  # Reference type values
  else:
    clazz = obj.__class__
    qualname = ".".join([clazz.__module__, clazz.__name__])
    
    # Restrict the circle of competence to Scikit-Learn classes
    if not (qualname.startswith("_abc.") or qualname.startswith("sklearn.")):
      raise ValueError(qualname)
    
    sum = object().__sizeof__() # Empty object
    names = get_instance_attrs(obj)
    if names:
      if verbose:
        print("| Attribute | `type(v)` | `deep_sklearn_sizeof(v)` |")
        print("|---|---|---|")
      for name in names:
        v = getattr(obj, name)
        v_type = type(v)
        v_sizeof = deep_sklearn_sizeof(v, verbose = False)
        sum += v_sizeof
        if verbose:
          print("| {} | {} | {} |".format(name, v_type, v_sizeof))
    return sum

In [21]:
import numpy as np
from sklearn.linear_model import LinearRegression
X, y = make_regression(n_samples=1000, n_features=10, n_informative=4, random_state=42)
model = LinearRegression()
model.fit(X,y)


print(deep_sklearn_sizeof(model, verbose=True))

| Attribute | `type(v)` | `deep_sklearn_sizeof(v)` |
|---|---|---|
| _abc_impl | <class '_abc._abc_data'> | 16 |
| _estimator_type | <class 'str'> | 58 |


ValueError: builtins.dict

In [ ]:
model = models.resnet18()
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2


print('model size: {:.3f}MB'.format(size_all_mb))


model size: 44.629MB